#### Алина, привет! 

Меня зовут Максим Попов, и я буду проверять твой проект. Давай будем общаться на «ты». Но если тебе комфортнее на «вы», обязательно сообщи мне об этом. 
По ходу тетрадки ты найдешь комментарии к проекту, они будут в отдельных ячейках Markdown с заголовком «Комментарий ревьюера». Постарайся учесть эти комментарии для дальнейших проектов.

---

Я буду отмечать комментарии цветом и примечанием. В конце заголовка комментария будет указан номер итерации проверки. Пожалуйста, не удаляй и не правь мои комментарии

Критично - требует исправления, влияет на удачное выполнение проекта

Рекомендация - комментарий является рекомендацией или советом. Mожешь учесть их при выполнении будущих заданий. На твое усмотрение

Отлично - Так отмечены удачные решения и подходы, на которые стоит опираться в будущих проектах

<div class="alert alert-block alert-danger">
    

### Комментарий от ревьюера
Критично

Такой комментарий нужно исправить обязательно, он критически влияет на удачное выполнение проекта.
</div>
    
---

<div class="alert alert-block alert-warning">
    

### Комментарий от ревьюера
Рекомендация

Такой комментарий является рекомендацией или советом. Mожешь учесть их при выполнении будущих заданий. На твое усмотрение. 
</div>

---

<div class="alert alert-block alert-success">
    

### Комментарий от ревьюера
Отлично
    
Так отмечены удачные решения и подходы, на которые стоит опираться в будущих проектах
</div>

<div class="alert alert-block alert-success">
    
    
### Комментарий от ревьюера v_1
    
Алина, отличная работа, но следующие моменты нужно подправить:
    
1. Таблицы-словари
2. Вывод по предобработке
    
Подробные комментарии по ходу проекта
   
Желтые комментарии-рекомендации на твое усмотрение.
    
Если есть вопросы ко мне, можешь оставить их в отдельном комментарии
</div>

<div class="alert alert-block alert-success">
 
    
### Комментарий от ревьюера v_2
    
Алина, привет!
    
Отлично, все критичные комментарии отработаны!
    
У тебя получился отличный проект, успеха в дальнейшем обучении!
</div>

# Исследование надежности заемщиков.

### Шаг 1. Обзор данных <a id="data_overview"></a>

<div class="alert alert-block alert-info">

  <b>Описание проекта</b>

  <b>Цель данного проекта:</b> разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
    
  <b>Входные данные:</b> статистика от банка о платёжеспособности клиентов.
</div>

<div class="alert alert-block alert-success">
    

### Комментарий от ревьюера v_1
Отлично
    
Молодец, что в начале проекта знакомишь с ним, так работа приобретает структуру и лучше воспринимается.
    
Но не все цели перечислены
</div>

In [92]:
import pandas as pd
data = pd.read_csv("/datasets/data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert alert-block alert-warning">
    

### Комментарий от ревьюера v_1
Рекомендация

Все используемые в проекте библиотеки следует импортировать в отдельной ячейке

</div>

In [93]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<div class="alert alert-block alert-info">
    Всего в данном проекте представлены данные об 21525 клиентах с 12 признаками, среди которых есть количественные и категориальные признаки. Признаки представлены данными нескольких типов: int, object, float.
</div>

<div class="alert alert-block alert-success">
    

### Комментарий от ревьюера v_1
Отлично
    
Отличный первичный взгляд на данные, идешь дальше
</div>

### Шаг 2.1 Заполнение пропусков  <a id="filling"></a>

In [94]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

<div class="alert alert-block alert-info">

  <b>Описание пропущенных значений</b>

  Были пропущены значения в столбце общего трудового стажа в днях, а также в столбце с данными о ежемесячном доходе.
</div>

In [95]:
empty_days_employed = data['days_employed'].isnull().sum()
empty_total_income = data['total_income'].isnull().sum()
total_value = len(data)
share_days_employed = empty_days_employed / total_value
share_total_income = empty_total_income / total_value
print(f"Доля пропущенных значений для столбца days_employed: {share_days_employed:.1%}")
print(f"Доля пропущенных значений для столбца total_income: {share_total_income:.1%}")

Доля пропущенных значений для столбца days_employed: 10.1%
Доля пропущенных значений для столбца total_income: 10.1%


<div class="alert alert-block alert-info">
    Исследовав столбец days_employed, я сделала предположение о том, что отрицательные значения представляют собой данные о количестве отработанных дней, которые необходимо привести к положительным, а также присутствуют значения в диапазоне 300000-401755, скорее всего это аномалии, либо данные, представленные не в днях. Для того, чтобы пропуски заполнять медианным значением необходимо для начала отрицательные значения привести к положительным.
</div>

In [96]:
#заменяем пропущенные значения в столбцу days_employed
data['days_employed'] = abs(data['days_employed'])
days_employed_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_median)

In [97]:
#заменяем пропущенные значения в столбце total_income
total_income_median = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(total_income_median)

In [98]:
#Проверим еще раз данные на пропуски, убедимся, что их сейчас нет
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<div class="alert alert-block alert-info">

  <b>Возможные причины появления пропусков</b>
    
  Возможные причины появления пропусков в данных:
    
  -ошибки, связанные с человеческим фактором
    
  -технологические ошибки
    
  Как я понимаю, пропуски могут быть также намеренными. К примеру, в данном случае видим, что пропущенные значения в столбцах  с трудовым стажем и ежемесячным доходом равны, при чем количество пропусков для данных столбцов одинаковое. Это говорит о том, что скорее всего данные клиенты не работали, соответственно не указан и их доход. Но также не исключается причина пропуска информации по части клиентов в связи с ошибками.
    
   
        
    
</div>

<div class="alert alert-block alert-info">

  <b>Описание пропущенных значений</b>
  
  Заполнение пропусков медианным значением для количественных переменных является оптимальным, поскольку среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. 

 
</div>
  
    


<div class="alert alert-block alert-success">

    
### Комментарий от ревьюера v_1
Отлично

Все верно, при таком разбросе значений медиана лучше всего характеризует распределение. Но все же замена общей медианой не самое оптимальное решение, можно заполнить медианой по группе, например, по типу занятости. По ссылке методичка по заполнению пропусков, рекомендую ознакомиться
    
https://drive.google.com/file/d/1Xgnl_LnmsGAhuPz6RzAzMTi9VvRBfHzp/view?usp=sharing
        
</div>

### Шаг 2.2 Проверка данных на аномалии и исправления. <a id="anomalies"></a>

<div class="alert alert-block alert-info">

  <b>Аномалии в столбце children</b>
  
  В столбце children аномальными являются значения 20 и -1. Думаю отрицательные значения означают, что знак минус был проставлен случайно, и нужно его убрать. А значение 20 можно заменить на среднее по данному столбцу.
    

 
</div>
  

In [99]:
#Просматриваем уникальные значения
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [100]:
#Избавимся от отрицательных значений
data['children'] = abs(data['children'])

In [101]:
#Создадим функцию для замены аномальных значений (20 детей)
def anomalies_children(child):
    if child == 20:
        return value_for_replacement
    
    return child

In [102]:
#Определим значение для замены
value_for_replacement = round(data['children'].mean())

In [103]:
#Методом apply применим объявленную выше функцию к данному столбцу
data['children'] = data['children'].apply(anomalies_children)

<div class="alert alert-block alert-warning">
    

### Комментарий от ревьюера v_1
Рекомендация

20 напрашивается заменить на 2
</div>

In [104]:
#Проверим признак children на наличие аномалий
data['children'].value_counts()

0    14149
1     4941
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

<div class="alert alert-block alert-info">

  <b>Аномалии в столбце days_employed</b>
    
  Ранее отрицательные значения были заменены на положительные. Однако было выявлено, что 3445 строки содержат данные в диапазоне от 328728,7 и выше. Я делаю предположение о том, что эти данные представлены в часах. И их необходимо заменить на данные в днях, разделив на 24.
  
</div>    

In [105]:
#Просмотрели, сколько строк с высокими значениями
high_values_days = data['days_employed'].sort_values(ascending=False).reset_index(drop=True)
high_values_days[:3447]

0       401755.400475
1       401715.811749
2       401675.093434
3       401674.466633
4       401663.850046
            ...      
3442    328771.341387
3443    328734.923996
3444    328728.720605
3445     18388.949901
3446     17615.563266
Name: days_employed, Length: 3447, dtype: float64

In [106]:
#Создали функцию для перевода данных из часов в дни
def anomalies_days(hours):
    if hours > 300000:
        return hours / 24
    
    return hours

In [107]:
#Заменили значения в столбце days_employed
data['days_employed'] = data['days_employed'].apply(anomalies_days)

In [108]:
#Разделим полученные значения на среднее количество рабочих дней в России и сравним с возрастом. 
#Для этого создадим в датафрейме столбец со стажем в годах
data['years_employed'] = data['days_employed'] / 250

In [109]:
#Получили, что у нас 2072 записи, где стаж в годах больше возраста. В большинстве случаев - это пенсионеры.
#Заменим эти выделяющиеся значения на медианный стаж. Для этого опять создадим функцию
days_median = data['days_employed'].median()
def anomalies_years(row):
    if row['dob_years'] < row['years_employed']:
        return days_median
    
    return row['days_employed']    

In [110]:
#Применим функцию ко всем строкам в столбце days_employed, заменив аномальные значения медианными
data['days_employed'] = data.apply(anomalies_years, axis=1)

<div class="alert alert-block alert-info">
  <b>Аномалии в столбце dob_years</b>
   
   Было выявлено, что присутствуют записи с значением 0 в данном столбце. Однако заполнять средним или медианным значением, по моему мнению, неправильно. Нужно посмотреть на категорию income_type данного клиента. Например, для пенсионеров заполнить средний возраст по группе пенсионеров и так далее.
</div>

In [111]:
#Просмотрим значения столбца dob_years в отсортированном порядке
for age in sorted(data['dob_years'].unique()):
    print(age, end=' ')

0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 

In [112]:
#Смотрим, какие категории есть для записей с аномальными значениями в столбце dob_years
data[data['dob_years'] == 0]['income_type'].unique()

array(['пенсионер', 'сотрудник', 'компаньон', 'госслужащий'], dtype=object)

In [113]:
#Рассчитаем средние для этих категорий
retiree_mean_age = data[(data['dob_years'] != 0) & (data['income_type'] == 'пенсионер')]['dob_years'].mean()
collaborator_mean_age = data[(data['dob_years'] != 0) & (data['income_type'] == 'сотрудник')]['dob_years'].mean()
companion_mean_age = data[(data['dob_years'] != 0) & (data['income_type'] == 'компаньон')]['dob_years'].mean()
civilservant_mean_age = data[(data['dob_years'] != 0) & (data['income_type'] == 'госслужащий')]['dob_years'].mean()

<div class="alert alert-block alert-warning">
    

### Комментарий от ревьюера v_1
Рекомендация

Такие действия следует проводить в цикле, сразу же меняя значения. Как это сделать есть в методичке, ссылку на которую ранее оставлял
</div>

In [114]:
#Определим функцию для замены аномальных значений в столбце dob_years
def anomalies_age(row):
    if row['dob_years'] == 0:
        if row['income_type'] == 'пенсионер':
            return int(retiree_mean_age)
        if row['income_type'] == 'сотрудник':
            return int(collaborator_mean_age)
        if row['income_type'] == 'компаньон':
            return int(companion_mean_age)
        if row['income_type'] == 'госслужащий':
            return int(civilservant_mean_age)
    
    return row['dob_years']    

In [115]:
#Вызовем функцию
data['dob_years'] = data.apply(anomalies_age, axis=1)

In [116]:
#Проверим, что нет аномальных значений с возрастом 0
for age in sorted(data['dob_years'].unique()):
    print(age, end=' ')

19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 

<div class="alert alert-block alert-info">
  Аномалии в столбце education не выявлены. 

</div>

In [117]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

<div class="alert alert-block alert-info">
  Аномалии в столбце education_id не выявлены.

</div>

In [118]:
data['education_id'].unique()

array([0, 1, 2, 3, 4])

<div class="alert alert-block alert-info">
  Аномалии в столбце family_status не выявлены.  

</div>

In [119]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

<div class="alert alert-block alert-info">
  Аномалии в столбце family_status_id не выявлены. 
</div>

In [120]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

<div class="alert alert-block alert-info">
  <b>Аномалии в столбце gender</b>
  
  В данном столбце выявлено одно аномальное значение XNA. По представленным данным его сложно исправить. Строку с данным значение по столбцу gender можно исключить из выборки, по моему мнению.

</div>

In [121]:
#Посмотрим на уникальные значения в столбце gender
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [122]:
#Посмотрим, сколько строк с аномальным значением в строке gender
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости,9.434402


In [123]:
#Удалим данную строку из данных
data = data[data['gender'] != 'XNA'].reset_index(drop=True)

In [124]:
#Проверяем, что количество записей стало в таблице меньше на одну и индексация произведена правильно
data.index

RangeIndex(start=0, stop=21524, step=1)

<div class="alert alert-block alert-info">
  Аномалии в столбце income_tupe не выявлены. 

</div>

In [125]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

<div class="alert alert-block alert-info">
  Аномалии в столбце debt не выявлены. 
</div>

In [126]:
data['debt'].unique()

array([0, 1])

<div class="alert alert-block alert-info">
  Аномалии в столбце total_income не выявлены. 
</div>

In [127]:
data['debt'].unique()

array([0, 1])

<div class="alert alert-block alert-info">
  Аномалии в столбце purpose не выявлены. 
</div>

In [128]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

<div class="alert alert-block alert-info">

  
  Таким образом, аномалии были обнаружены в нескольких столбцах, как описано выше. Аномалии могут возникать как ошибка в результате человеческого фактора, также в результате технолгической ошибки. Данные могут быть представлены в разных единицах измерения, могут просто отсутствовать. С аномалиями нужно разбираться, для того, чтобы правильно ответить на вопрос поставленной задачи.

</div>

<div class="alert alert-block alert-success">

    
### Комментарий от ревьюера v_1
Отлично

Отлично, аномалий не осталось 👍
        
</div>

In [129]:
#Перезапишем таблицу без последнего столбца years_employed, который использовали для обработки аномальных значений
data = data.loc[:,'children':'purpose']

### Шаг 2.3. Изменение типов данных. <a id="types"></a>

In [130]:
#Заменяем тип данных на целочисленный в столбце total_income
data['total_income'] = data['total_income'].astype('int')

<div class="alert alert-block alert-success">

    
### Комментарий от ревьюера v_1
Отлично

Отлично, теперь данные хранятся в правильно виде 👍🏻
    
</div>

### Шаг 2.4. Удаление дубликатов. <a id="delete"></a>

<div class="alert alert-block alert-info">
Поиск явных дубликатов
</div>

In [131]:
#Проверим количество дубликатов в таблице
data.duplicated().sum()

54

In [132]:
#Удалим дубликаты
data = data.drop_duplicates().reset_index(drop=True)

<div class="alert alert-block alert-warning">
    

### Комментарий от ревьюера v_1
Рекомендация

При удалении дубликатов следует обновить индексы
</div>

<div class="alert alert-block alert-info">

  <b>Вопрос по комментарию</b>
    А разве я не сделала обновление индексов в ячейке выше?   
    


</div>

<div class="alert alert-block alert-success">

    
### Комментарий от ревьюера v_2

Сори, действительно сделала...
    
</div>

In [133]:
#Проверим результат
data.duplicated().sum()

0

<div class="alert alert-block alert-info">
    Поиск неявных дубликатов
</div>

In [134]:
#Просматриваем уникальные значения в поле education
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [135]:
#Приводим к нижнему регистру, чтобы избежать дублирующихся записей
data['education'] = data['education'].str.lower()

<div class="alert alert-block alert-warning">
    

### Комментарий от ревьюера v_1
Рекомендация

После обработки данных значений следует еще раз проверить на дубликаты датасет
</div>

In [136]:
#Проверка остальных столбцов
data['family_status'].unique()
data['gender'].unique()
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

<div class="alert alert-block alert-info">
    Для определения явных дубликатов можно использовать метод duplicted() и метод sum() для подсчета дубликатов.     
    Для удаления дубликатов из таблицы, то есть повторяющихся записей используется метод drop_duplicates().    
    Для обнаружения неявных дубликатов можно использовать методы unique(), а также метод value_counts().
    Дубликаты могут возникать по причине человеческого фактора, а также по причине технологического фактора.   
</div>

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма. <a id="decomposition"></a>

In [137]:
#Создание новых датафреймов
education = data[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
family_status = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

#Удалим столбцы education и family_status из исходного датафрейма
data.drop('education', axis=1, inplace=True)
data.drop('family_status', axis=1, inplace=True)

<div class="alert alert-block alert-danger">
    

### Комментарий от ревьюера v_1
Критично

В education надо было дубликаты дропнуть, у тебя не словари получились, а все данные из таблицы
    
</div>

<div class="alert alert-block alert-info">
<b>Исправила</b>
</div>

<div class="alert alert-block alert-success">

    
### Комментарий от ревьюера v_2

👍🏻
    
</div>

### Шаг 2.6. Категоризация дохода. <a id="categorization"></a>

In [47]:
#Создадим функцию для категоризации дохода
def income_category(income):
    if income < 30000:
        return 'E'
    if 30001 <= income < 50000:
        return 'D'
    if 50001 <= income < 200000:
        return 'C'
    if 200001 <= income < 1000000:
        return 'B'
    
    return 'A'    

In [48]:
#Создадим в исходном датафрейме дополнительный столбец
data['total_income_category'] = data['total_income'].apply(income_category)

### Шаг 2.7. Категоризация целей кредита. <a id="categorization2"></a>

In [49]:
#создадим списки с одинаковыми по смыслу целями
realty = ['покупка жилья', 'операции с жильем', 'покупка жилья для семьи', 'жилье', 'покупка своего жилья', 'покупка жилья для сдачи', 'ремонт жилью']
car = ['приобретение автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'автомобили', 'сделка с подержанным автомобилем', 'автомобиль', 'свой автомобиль', 'сделка с автомобилем', 'на покупку автомобиля', 'покупка недвижимости', 'покупка коммерческой недвижимости', 'покупка жилой недвижимости', 'строительство собственной недвижимости', 'недвижимость', 'строительство недвижимости', 'операции с коммерческой недвижимостью', 'строительство жилой недвижимости', 'операции со своей недвижимостью', 'операции с недвижимостью']
education = ['дополнительное образование', 'образование', 'заняться образованием', 'получение образования', 'получение дополнительного образования', 'получение высшего образования', 'профильное образование', 'высшее образование', 'заняться высшим образованием']
wedding = ['сыграть свадьбу','на проведение свадьбы', 'свадьба']

#Определяем функцию
def purpose_duplicates(purpose):
    if purpose in realty:
        return 'операции с недвижимостью'
    if purpose in car:
        return 'операции с автомобилем'
    if purpose in education:
        return 'получение образования'
    if purpose in wedding:
        return 'проведение свадьбы'
    
    return purpose

<div class="alert alert-block alert-warning">
    

### Комментарий от ревьюера v_1
Рекомендация
    
Вот так делать не надо, слишком усложняешь процесс. Надо было категоризировать по вхождении строки в строку. Например, если "свадьб" in там-то, то возвращаешь на свадьбу и т. п.
</div>

In [50]:
#Создаем новый столбец при помощи метода apply
data['purpose_category'] = data['purpose'].apply(purpose_duplicates)

In [51]:
#Проверяем результат
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

<div class="alert alert-block alert-success">

    
### Комментарий от ревьюера v_1
Отлично
    
Отлично, с категоризацией разобрались 👍🏻
        
</div>

<div class="alert alert-block alert-danger">
    

### Комментарий от ревьюера v_1
Критично

Не хватает резюмирующего вывода по шагу предобработки данных
</div>

<div class="alert alert-block alert-info">
Предобработка данных включала несколько этапов. На первом этапе были рассмотрены пропущенные значения. Пропуски в столбцах с количественными перменными были заполнены медианным значением, поскольку среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. Далее данные были проверены на аномалии, которые были обработаны. Аномалии могут возникать как ошибка в результате человеческого фактора, также в результате технолгической ошибки. Данные могут быть представлены в разных единицах измерения, могут просто отсутствовать. С аномалиями нужно разбираться, для того, чтобы правильно ответить на вопрос поставленной задачи.
Далее данные были преобразованы к необходимому типу, также были удалены явные и неявные дубликаты. Категоризация дохода была проивзедена в соответсвии с размером дохода. Цели кредита были категоризированы в зависимости от описания.
</div>

<div class="alert alert-block alert-success">

    
### Комментарий от ревьюера v_2

👍🏻
    
</div>

### Ответы на вопросы. <a id="questions"></a>

<div class="alert alert-block alert-info">
        Для ответа на поставленные вопросы необходимо сначала сгруппировать таблицу по соответствующему полю и посчитать общее количество записей для каждой группы из поля debt. Значение 0 для поля debt означает, что долгов по кредиту не было. Запишем сумму по двум группам в отдельный столбец total, а дальше посчитаем отношение столбца debt 0 к столбцу total. Получим значения в процентах, далее отсортируем по убыванию поля part.
</div>    


##### Вопрос 1:

<div class="alert alert-block alert-info">
<b>Зависимость между количеством детей и вовратом кредита в срок</b>
</div>    

In [52]:
#создаем сводную таблицу
data_pivot = data.pivot_table(index=['children'], columns='debt', values='dob_years', aggfunc='count')

#заполняем пропуски 0 для дальнейшего корректного расчета
data_pivot = data_pivot.fillna(0)

#записываем в столбец total сумму столбцов 0 и 1
data_pivot['total'] = data_pivot[0] + data_pivot[1]

#записываем в столбец part отношение столбца 0 к общей сумме для каждой группы в процентах
data_pivot['part'] = (data_pivot[0] / data_pivot['total']) * 100 

#Выводим таблицу в остортированном по полю part виде
data_pivot.sort_values(by='part', ascending=False)

debt,0,1,total,part
children,,,,
5,9.0,0.0,9.0,100.000000
0,13043.0,1063.0,14106.0,92.464200
3,303.0,27.0,330.0,91.818182
1,4479.0,453.0,4932.0,90.815085
2,1858.0,194.0,2052.0,90.545809
4,37.0,4.0,41.0,90.243902


##### Вывод 1:

<div class="alert alert-block alert-info">
Клиенты, имеющие 5 детей, всегда возвращают кредит в срок. Однако, они составляют незначительную долю в выборке. Наиболее многочисленной группой является группа без детей, а также у этой группы самый высокий процент возврата кредита в срок после 5-детных. Наименьший процент возврата кредита в срок имеет группа с 4-мя детьми.
</div> 

##### Вопрос 2:

<div class="alert alert-block alert-info">
<b>Зависимость между семейным положением и вовратом кредита в срок</b>
</div>

In [53]:
#создаем сводную таблицу
data_pivot = data.pivot_table(index=['family_status_id'], columns='debt', values='dob_years', aggfunc='count')

#записываем в столбец total сумму столбцов 0 и 1
data_pivot['total'] = data_pivot[0] + data_pivot[1]

#записываем в столбец part отношение столбца 0 к общей сумме для каждой группы в процентах
data_pivot['part'] = (data_pivot[0] / data_pivot['total']) * 100 

#Выводим таблицу в остортированном по полю part виде
data_pivot.sort_values(by='part', ascending=False)

debt,0,1,total,part
family_status_id,,,,
2,896,63,959,93.430657
3,1110,85,1195,92.887029
0,11413,931,12344,92.457874
1,3774,388,4162,90.677559
4,2536,274,2810,90.249110


In [54]:
family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


##### Вывод 2:

<div class="alert alert-block alert-info">

Клиенты, которые являются вдовцами, имеют самый высокий процент возврата кредита в срок. Наиболее многочисленная группа женатых находятся в середине.  Наименьший процент возврата кредита в срок имеет группа неженатых.
</div>

##### Вопрос 3:

<div class="alert alert-block alert-info">
  
<b>Зависимость между уровнем дохода и возвратом кредита в срок</b>    
</div>

In [55]:
#создаем сводную таблицу
data_pivot = data.pivot_table(index=['total_income_category'], columns='debt', values='dob_years', aggfunc='count')

#записываем в столбец total сумму столбцов 0 и 1
data_pivot['total'] = data_pivot[0] + data_pivot[1]

#записываем в столбец part отношение столбца 0 к общей сумме для каждой группы в процентах
data_pivot['part'] = (data_pivot[0] / data_pivot['total']) * 100 

#Выводим таблицу в остортированном по полю part виде
data_pivot.sort_values(by='part', ascending=False)

debt,0,1,total,part
total_income_category,,,,
D,329,21,350,94.000000
B,4684,356,5040,92.936508
A,23,2,25,92.000000
C,14673,1360,16033,91.517495
E,20,2,22,90.909091


##### Вывод 3:

<div class="alert alert-block alert-info">
Клиенты с уровнем дохода 30001 - 50000 являются наиболее добросовестными и возвращают кредит в срок. Самыми недобросовестными являются клиенты с доходом менее 30000. Самая многочисленная группа с доходом 50001 - 200000 находится на предпоследнем месте.
</div>

##### Вопрос 4:

<div class="alert alert-block alert-info">
    
<b>Влияние целей кредита на возврат в срок</b>
</div>

In [56]:
#создаем сводную таблицу
data_pivot = data.pivot_table(index=['purpose_category'], columns='debt', values='dob_years', aggfunc='count')

#записываем в столбец total сумму столбцов 0 и 1
data_pivot['total'] = data_pivot[0] + data_pivot[1]

#записываем в столбец part отношение столбца 0 к общей сумме для каждой группы в процентах
data_pivot['part'] = (data_pivot[0] / data_pivot['total']) * 100 

#Выводим таблицу в остортированном по полю part виде
data_pivot.sort_values(by='part', ascending=False)

debt,0,1,total,part
purpose_category,,,,
операции с недвижимостью,4153,308,4461,93.095718
проведение свадьбы,2149,186,2335,92.034261
операции с автомобилем,9783,877,10660,91.772983
получение образования,3644,370,4014,90.782262


##### Вывод 4:

<div class="alert alert-block alert-info">

Клиенты, цель которых операции с недвижимостью, имеет самый высокий процент возврата кредита в срок по сравнению с другими группами. Наименьший процент возврата у клиентов, которые берут кредит на получение образования.    
</div>    

<div class="alert alert-block alert-success">

    
### Комментарий от ревьюера v_1
    
Отлично

Отличный шаг, все расчеты и выводы верны, молодец, что не пренебрегаешь малыми процентами
    
Рекомендация по улучшению
    
У нас задолженность представлена в числах, 1 - есть задолженность, 0 - нет задолженности. Это значит, что среднее арифметическое - это и будет доля должников.    
    
Можно построить сводную таблицу, посчитав в ней три агрегирующие функции - количество, сумму и среднее, и решение будет в одну строчку. 

- количество (count) - всего клиентов в группе  
- сумма (sum) - количество должников
- среднее (mean) - доля должников
    
Рекомендую почитать про сводные таблицы тут, и вообще ресурс отличный, кидай его в закладки
    
http://dfedorov.spb.ru/pandas/%D0%A1%D0%B2%D0%BE%D0%B4%D0%BD%D0%B0%D1%8F%20%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0%20%D0%B2%20pandas.html
</div>

## Общий вывод:

Таким образом, в данном проекте предобработали полученные данные, [заполнили пропуски](#filling), [разобрались с аномалиями](#anomilies), [изменили тип данных](#types), [обработали явные и неявные дубликаты](#delete), [осуществили категоризацию данных](#categorization). А также ответили на [поставленные вопросы](#questions).
</div>

<div class="alert alert-block alert-warning">
    

### Комментарий от ревьюера v_1
Рекомендация
    
Все-таки в выводе нужны результаты, а не ссылки ))
</div>